# Step 1: Import Libraries

In [1]:
import importlib
import subprocess
import torch

# Step 2: Clone Repo

In [2]:
!git clone https://github.com/M-3LAB/open-iad.git
%cd open-iad

Cloning into 'open-iad'...
remote: Enumerating objects: 9740, done.
remote: Counting objects: 100% (671/671), done.
remote: Compressing objects: 100% (224/224), done.
remote: Total 9740 (delta 536), reused 527 (delta 445), pack-reused 9069 (from 1)
Receiving objects: 100% (9740/9740), 94.23 MiB | 14.53 MiB/s, done.
Resolving deltas: 100% (5860/5860), done.
/content/open-iad


In [3]:
!ls

arch	      checkpoints    data_io  loss_function  metric  optimizer	README.md	  tools
augmentation  configuration  dataset  main.py	     models  paradigms	requirements.txt


# Step 3: Install Dependencies

In [4]:
def is_installed(lib_name):
    try:
        importlib.import_module(lib_name)
        return True
    except ImportError:
        return False

In [5]:
def get_import_name(package_name):
    mapping = {
        "opencv_python": "cv2",
        "scikit_image": "skimage",
        "scikit_learn": "sklearn",
        "PyYAML": "yaml",
        "faiss_gpu": "faiss",
        "pytorch_msssim": "pytorch_msssim",
    }
    return mapping.get(package_name, package_name.replace("-", "_"))

In [6]:
missing_packages = []
print("Checking requirements...")

with open("requirements.txt", "r") as file:
    for line in file:
        line = line.strip()
        if line.startswith("#") or not line:
            continue

        package_name = line.split("==")[0]
        import_name = get_import_name(package_name)

        if is_installed(import_name):
            print(f"✅ {package_name} is already installed.")
        else:
            print(f"❌ {package_name} is missing.")
            missing_packages.append(line)

Checking requirements...
✅ rich is already installed.
✅ opencv_python is already installed.
✅ scipy is already installed.
✅ tqdm is already installed.
✅ PyYAML is already installed.
❌ open3d is missing.
❌ scikit-image is missing.
❌ scikit-learn is missing.
❌ munch is missing.
❌ ninja is missing.
✅ matplotlib is already installed.
✅ tifffile is already installed.
✅ imgaug is already installed.
❌ kornia is missing.
❌ torchmetrics is missing.
❌ faiss-gpu is missing.
✅ timm is already installed.
❌ efficientnet_pytorch is missing.
❌ FrEIA is missing.
❌ ignite is missing.
✅ einops is already installed.
❌ torchprofile is missing.
❌ pytorch-msssim is missing.


In [7]:
if missing_packages:
    print("\nInstalling missing packages...")
    failed_packages = []
    for package in missing_packages:
        try:
            print(f"Installing {package}...")
            subprocess.check_call(["pip", "install", package], timeout=300)
            print(f"✅ {package} installed successfully!")
        except subprocess.CalledProcessError as e:
            print(f"❌ Failed to install {package}. Error: {e}")
            failed_packages.append(package)
        except subprocess.TimeoutExpired:
            print(f"❌ Installation timed out for {package}. Skipping.")
            failed_packages.append(package)

    if failed_packages:
        print("\nRetrying failed packages with default versions...")
        default_failed_packages = [pkg.split("==")[0] for pkg in failed_packages]
        for package in default_failed_packages:
            try:
                print(f"Installing {package} with default version...")
                subprocess.check_call(["pip", "install", package], timeout=300)
                print(f"✅ {package} installed successfully with default version!")
            except subprocess.CalledProcessError as e:
                print(f"❌ Failed to install {package} even with default version. Error: {e}")
            except subprocess.TimeoutExpired:
                print(f"❌ Installation timed out for {package} (default version). Skipping.")
else:
    print("\nAll required packages are already installed!")


Installing missing packages...
Installing open3d==0.15.2...
❌ Failed to install open3d==0.15.2. Error: Command '['pip', 'install', 'open3d==0.15.2']' returned non-zero exit status 1.
Installing scikit-image==0.19.2...
❌ Installation timed out for scikit-image==0.19.2. Skipping.
Installing scikit-learn==1.0.2...
❌ Failed to install scikit-learn==1.0.2. Error: Command '['pip', 'install', 'scikit-learn==1.0.2']' returned non-zero exit status 1.
Installing munch==2.5.0...
✅ munch==2.5.0 installed successfully!
Installing ninja==1.10.2.3...
✅ ninja==1.10.2.3 installed successfully!
Installing kornia==0.6.5...
✅ kornia==0.6.5 installed successfully!
Installing torchmetrics==0.9.1...
✅ torchmetrics==0.9.1 installed successfully!
Installing faiss-gpu==1.7.2...
❌ Failed to install faiss-gpu==1.7.2. Error: Command '['pip', 'install', 'faiss-gpu==1.7.2']' returned non-zero exit status 1.
Installing efficientnet_pytorch==0.7.1...
✅ efficientnet_pytorch==0.7.1 installed successfully!
Installing Fr

# Step 4: Verify GPU Access

In [8]:
"""
If torch.cuda.is_available() returns True then we can ues the GPU
"""

print(torch.cuda.is_available())

True


In [16]:
"""
If torch.cuda.device_count() returns 0, it means no GPU is available.
If it returns 1, the only valid GPU ID is 0.
"""

print(f"Available GPUs: {torch.cuda.device_count()}")
print(f"Current GPU: {torch.cuda.current_device()}")
print(f"GPU Name: {torch.cuda.get_device_name(0)}")

Available GPUs: 1
Current GPU: 0
GPU Name: Tesla T4


# Step 5: Change some part of the code for correctness

In [9]:
device_py_content = """
import socket
import fcntl
import struct
from configuration.registration import server_data


def get_ip_address(ifname):
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        info = struct.pack('256s', bytes(ifname[:15], 'utf-8'))
        result = socket.inet_ntoa(fcntl.ioctl(s.fileno(), 0x8915, info)[20:24])
        return result
    except (OSError, ImportError):
        # Handle errors for environments like Colab
        print(f"Warning: Unable to get IP for interface '{ifname}'. Using '127.0.0.1' as default.")
        return '127.0.0.1'


def assign_service(moda='eno1'):
    ip = get_ip_address(moda)
    root_path = server_data.get(ip, "/content/data")  # Use default root path if IP is not in server_data

    return ip, root_path
"""

with open("/content/open-iad/configuration/device.py", "w") as f:
    f.write(device_py_content)

print("Updated device.py saved successfully!")

Updated device.py saved successfully!


In [10]:
registration_py_content = """
# setting
setting_name = ['vanilla', 'fewshot', 'semi', 'noisy', 'continual', 'transfer']

# add new dataset
dataset_name = {'_example': ('dataset._example', '_example', 'Example'),
                'mvtec2d': ('dataset.mvtec2d', 'mvtec2d', 'MVTec2D'),
                'mvtec2df3d': ('dataset.mvtec2df3d', 'mvtec2df3d', 'MVTec2DF3D'),
                'mvtecloco': ('dataset.mvtecloco', 'mvtecloco', 'MVTecLoco'),
                'mpdd': ('dataset.mpdd', 'mpdd', 'MPDD'),
                'btad': ('dataset.btad', 'btad', 'BTAD'),
                'mtd': ('dataset.mtd', 'mtd', 'MTD'),
                'mvtec3d': ('dataset.mvtec3d', 'mvtec3d', 'MVTec3D'),
                'visa': ('dataset.visa', 'visa', 'VisA'),
                'dagm': ('dataset.dagm', 'dagm', 'DAGM'),
                'coad': ('dataset.coad', 'coad', 'COAD'),
                }

# add new model
model_name = {'_example': ('arch._example', '_example', 'Example'),
              '_patchcore': ('arch._patchcore', '_patchcore', 'PatchCore'),
              'patchcore': ('arch.patchcore', 'patchcore', 'PatchCore'),
              'padim': ('arch.padim', 'padim', 'PaDim'),
              'csflow': ('arch.csflow', 'csflow', 'CSFlow'),
              'dne': ('arch.dne', 'dne', 'DNE'),
              'draem': ('arch.draem', 'draem', 'DRAEM'),
              'igd': ('arch.igd', 'igd', 'IGD'),
              'dra': ('arch.dra', 'dra', 'DRA'),
              'devnet': ('arch.devnet', 'devnet', 'DevNet'),
              'favae': ('arch.favae', 'favae', 'FAVAE'),
              'fastflow': ('arch.fastflow', 'fastflow', 'FastFlow'),
              'cfa': ('arch.cfa', 'cfa', 'CFA'),
              'reverse': ('arch.reverse', 'reverse', 'REVERSE'),
              'spade': ('arch.spade', 'spade', 'SPADE'),
              'stpm': ('arch.stpm', 'stpm', 'STPM'),
              'cutpaste': ('arch.cutpaste', 'cutpaste', 'CutPaste'),
              'graphcore': ('arch.graphcore', 'graphcore', 'GraphCore'),
              'simplenet': ('arch.simplenet', 'simplenet', 'SimpleNet'),
              'softpatch': ('arch.softpatch', 'softpatch', 'SoftPatch'),
              }

# server config, ip: dataset root path
server_data = {'127.0.0.1': '/content/open-iad/data',
               '172.18.36.108': '/ssd2/m3lab/data/open-ad',
               '172.18.36.107': '/ssd-sata1/wjb/data/open-ad',
              }
"""

with open("/content/open-iad/configuration/registration.py", "w") as f:
    f.write(registration_py_content)

print("Updated registration.py saved successfully!")

Updated registration.py saved successfully!


# Step 6: Download the dataset

In [11]:
!wget https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420938113-1629952094/mvtec_anomaly_detection.tar.xz

--2025-02-12 17:31:16--  https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420938113-1629952094/mvtec_anomaly_detection.tar.xz
Resolving www.mydrive.ch (www.mydrive.ch)... 91.214.169.64
Connecting to www.mydrive.ch (www.mydrive.ch)|91.214.169.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5264982680 (4.9G) [application/x-xz]
Saving to: ‘mvtec_anomaly_detection.tar.xz’

mvtec_anomaly_detec 100%[===================>]   4.90G  17.4MB/s    in 4m 45s  

2025-02-12 17:36:02 (17.6 MB/s) - ‘mvtec_anomaly_detection.tar.xz’ saved [5264982680/5264982680]



In [12]:
!mkdir /content/open-iad/data

In [13]:
!mkdir /content/open-iad/data/mvtec2d

In [14]:
!tar -xf /content/open-iad/mvtec_anomaly_detection.tar.xz -C /content/open-iad/data/mvtec2d

# Step 7: Run the algorithm

In [17]:
!python3 main.py -f -fe 1 -m reverse -n net_reverse -d mvtec2d -tid 0 -vid 0 -g 0

local ip: 127.0.0.1, root_path: /content/open-iad/data
---------------------
{
    'paradigm': 'c2d',
    'dataset': 'mvtec2d',
    'model': 'reverse',
    'net': 'net_reverse',
    'train_task_id': [0],
    'valid_task_id': [0],
    'fewshot': True,
    'fewshot_exm': 1,
    'gpu_id': '0',
    'vis': True
}
---------------------
{
    'dataset': 'mvtec2d',
    'root_path': '/content/open-iad/data',
    'data_path': '/content/open-iad/data/mvtec2d',
    'num_task': 15,
    'data_size': 256,
    'data_crop_size': 256,
    'mask_size': 256,
    'mask_crop_size': 256,
    'learning_mode': 'centralized',
    'vanilla': False,
    'semi': False,
    'semi_anomaly_num': 5,
    'semi_overlap': False,
    'ref_num': 5,
    'continual': False,
    'fewshot': True,
    'fewshot_exm': 1,
    'fewshot_num_dg': 4,
    'fewshot_data_aug': False,
    'fewshot_feat_aug': False,
    'fewshot_aug_type': ['normal'],
    'noisy': False,
    'noisy_ratio': 0.1,
    'noisy_overlap': False,
    'transfer': F

# Step 8: implement new way

In [ ]:
new_registration_py_content = """
# setting
setting_name = ['vanilla', 'fewshot', 'semi', 'noisy', 'continual', 'transfer']

# add new dataset
dataset_name = {'_example': ('dataset._example', '_example', 'Example'),
                'mvtec2d': ('dataset.mvtec2d', 'mvtec2d', 'MVTec2D'),
                'mvtec2df3d': ('dataset.mvtec2df3d', 'mvtec2df3d', 'MVTec2DF3D'),
                'mvtecloco': ('dataset.mvtecloco', 'mvtecloco', 'MVTecLoco'),
                'mpdd': ('dataset.mpdd', 'mpdd', 'MPDD'),
                'btad': ('dataset.btad', 'btad', 'BTAD'),
                'mtd': ('dataset.mtd', 'mtd', 'MTD'),
                'mvtec3d': ('dataset.mvtec3d', 'mvtec3d', 'MVTec3D'),
                'visa': ('dataset.visa', 'visa', 'VisA'),
                'dagm': ('dataset.dagm', 'dagm', 'DAGM'),
                'coad': ('dataset.coad', 'coad', 'COAD'),
                }

# add new model
model_name = {'_example': ('arch._example', '_example', 'Example'),
              '_patchcore': ('arch._patchcore', '_patchcore', 'PatchCore'),
              'patchcore': ('arch.patchcore', 'patchcore', 'PatchCore'),
              'padim': ('arch.padim', 'padim', 'PaDim'),
              'csflow': ('arch.csflow', 'csflow', 'CSFlow'),
              'dne': ('arch.dne', 'dne', 'DNE'),
              'draem': ('arch.draem', 'draem', 'DRAEM'),
              'igd': ('arch.igd', 'igd', 'IGD'),
              'dra': ('arch.dra', 'dra', 'DRA'),
              'devnet': ('arch.devnet', 'devnet', 'DevNet'),
              'favae': ('arch.favae', 'favae', 'FAVAE'),
              'fastflow': ('arch.fastflow', 'fastflow', 'FastFlow'),
              'cfa': ('arch.cfa', 'cfa', 'CFA'),
              'reverse': ('arch.reverse', 'reverse', 'REVERSE'),
              'spade': ('arch.spade', 'spade', 'SPADE'),
              'stpm': ('arch.stpm', 'stpm', 'STPM'),
              'cutpaste': ('arch.cutpaste', 'cutpaste', 'CutPaste'),
              'graphcore': ('arch.graphcore', 'graphcore', 'GraphCore'),
              'simplenet': ('arch.simplenet', 'simplenet', 'SimpleNet'),
              'softpatch': ('arch.softpatch', 'softpatch', 'SoftPatch'),
              'ahmadasadi': ('arch.ahmadasadi', 'ahmadasadi', 'AhmadAsadi'),
              }

# server config, ip: dataset root path
server_data = {'127.0.0.1': '/content/open-iad/data',
               '172.18.36.108': '/ssd2/m3lab/data/open-ad',
               '172.18.36.107': '/ssd-sata1/wjb/data/open-ad',
              }
"""

with open("/content/open-iad/configuration/registration.py", "w") as f:
    f.write(new_registration_py_content)

print("Updated registration.py saved successfully!")

In [ ]:
new_config_py_content = """
import argparse
# from asyncio import FastChildWatcher
from logging import root

__all__ = ['parse_arguments_main']

def parse_arguments_main():
    parser = argparse.ArgumentParser()
    ## learning paradigm
    parser.add_argument('--paradigm', '-p', type=str, default='c2d', choices=['c2d', 'c3d', 'f2d'])

    # ----------------------------- centralized learning ----------------------------- #
    parser.add_argument('--dataset', '-d', type=str, default='mvtec2d', choices=['_example', 'mvtec2d', 'mvtec3d', 'mpdd', 'mvtecloco', 'mtd',
                                                                              'btad', 'mvtec2df3d', 'visa', 'dagm', 'coad'])
    parser.add_argument('--model', '-m', type=str, default='softpatch', choices=['_example', '_patchcore', 'patchcore', 'csflow', 'dne',
        'draem', 'igd', 'cutpaste', 'devnet', 'dra', 'favae', 'padim', 'reverse', 'spade', 'fastflow', 'softpatch', 'cfa', 'stpm',
        'simplenet', 'softpatch', 'ahmadasadi'])
    parser.add_argument('--net', '-n', type=str, default='wide_resnet50', choices=['net_example', 'wide_resnet50', 'resnet18', 'net_csflow',
        'vit_b_16', 'net_draem', 'net_dra', 'net_igd', 'net_reverse', 'net_favae', 'net_fastflow', 'net_cfa', 'net_devnet',
        'vig_ti_224_gelu', 'net_ahmadasadi'])

    parser.add_argument('--root-path', '-rp', type=str, default=None)
    parser.add_argument('--data-path', '-dp', type=str, default=None)

    parser.add_argument('--train-task-id', '-tid', type=int, default=[0], nargs='+')
    parser.add_argument('--valid-task-id', '-vid', type=int, default=[0], nargs='+')
    parser.add_argument('--sampler-percentage', '-sp', type=float, default=None)

    # vanilla
    parser.add_argument('--vanilla', '-v', action='store_true', default=False)

    # semi-supervised
    parser.add_argument('--semi', '-s', action='store_true', default=False)
    parser.add_argument('--semi-anomaly-num', '-san', type=int, default=None)
    parser.add_argument('--semi-overlap', '-so', action='store_true', default=False)

    # continual
    parser.add_argument('--continual', '-c', action='store_true', default=False)

    # fewshot
    parser.add_argument('--fewshot', '-f', action='store_true', default=False)
    parser.add_argument('--fewshot-exm', '-fe', type=int, default=None)
    parser.add_argument('--fewshot-data-aug', '-fda', action='store_true', default=False)
    parser.add_argument('--fewshot-feat-aug', '-ffa', action='store_true', default=False)
    parser.add_argument('--fewshot-num-dg', '-fnd', type=int, default=None)
    parser.add_argument('--fewshot-aug-type', '-fat', default=None, nargs='+',
                        choices=['normal', 'rotation', 'scale', 'translate', 'flip', 'color_jitter', 'perspective'])

    # noisy label
    parser.add_argument('--noisy', '-z', action='store_true', default=False)
    parser.add_argument('--noisy-overlap', '-no', action='store_true', default=False)
    parser.add_argument('--noisy-ratio', '-nr', type=float, default=None)

    # transfer
    parser.add_argument('--transfer', '-t', action='store_true', default=False)
    parser.add_argument('--transfer-target-sample-num', '-ttn', type=int, default=None)

    # data augmentation type
    parser.add_argument('--train-aug-type', '-tag', default=None, choices=['normal', 'cutpaste'], help='data augmentation type')
    parser.add_argument('--valid-aug-type', '-vag', default=None, choices=['normal', 'cutpaste'], help='data augmentation type')

    # universal
    parser.add_argument('--gpu-id', '-g', type=str, default=None)
    parser.add_argument('--server-moda', '-sm', type=str, default=None, choices=['eno1', 'lo'])
    parser.add_argument('--num-epochs', '-ne', type=int, default=None)
    parser.add_argument('--seed', type=int, default=None)
    parser.add_argument('--debug', action='store_true', default=False)
    parser.add_argument('--vis', '-vis', action='store_true', default=True)
    parser.add_argument('--vis-em', action='store_true', default=False)

    parser.add_argument('--save-model', action='store_true', default=False)
    parser.add_argument('--load-model', action='store_true', default=False)
    parser.add_argument('--load-model-dir', type=str, default=None)

    # ----------------------------- federated learning ----------------------------- #
    parser.add_argument('--fed-aggregate-method', '-fam', type=str, default=None)
    parser.add_argument('--num-round', type=int, default=None)


    args = parser.parse_args()
    return args


"""

with open("/content/open-iad/configuration/config.py", "w") as f:
    f.write(new_config_py_content)

print("Updated registration.py saved successfully!")